In [4]:
import numpy as np
import matplotlib.pyplot as plt
#import tensorflow as tf
from scipy.special import expit
import time
import matplotlib.pyplot as plt

In [5]:
%matplotlib qt5

In [21]:
class BPNN():
    def __init__(self, etha, nodes, inputs, outputs):
        self.etha = etha
        self.hidden_nodes = nodes
        self.input = np.array(inputs)
        self.output = np.array(outputs)
        self.NN_structure()
        
    def sigmoid(self,x):
        #return 1/(np.exp(-x)+1) -->소수점때문에 overflow 현상 일어나서 scipy 모듈로 대체
        return expit(x)
    
    def sigmoid_prime(self,x):
        return self.sigmoid(x)*(1-self.sigmoid(x))
    
    def NN_structure(self):
        
        bias = np.ones(shape=(len(self.input),1))
        self.input = np.concatenate([self.input,bias],axis=1)
        inputs_num = len(self.input[0])
        outputs_num = len(self.output[0])
        
        #weight
        self.w1 = np.random.rand(inputs_num,self.hidden_nodes)
        self.w2 = np.random.rand(self.hidden_nodes+1,outputs_num)
        
        #Problem 3 weight
        #self.w1 = np.array([[-0.089,0.098],[0.028,-0.07],[0.092,-0.01]])
        #self.w2 = np.array([[0.056],[0.067],[0.016]])
        
    def run(self, iteration, verbose=False, Error_graph = False):
        t= time.time()
        y =[]
        for j in range(1,iteration+1):
            sum1 = 0
            sum2 = 0
            sum_Error = 0
            for i in range(len(self.input)):
                
                #calculation until hidden node
                net = np.matmul(self.input[i],self.w1)
                h = self.sigmoid(net)
                #add bias to hidden node
                h = np.concatenate([h,np.ones(1).T])
                
                #calculation until output
                o_net = np.matmul(h,self.w2)
                o = self.sigmoid(o_net)

                #Cost function
                Error = 0.5*sum((o-self.output[i])*(o-self.output[i]))
                
                #delta weight
                delta_w_2 = np.multiply((o-self.output[i]),self.sigmoid_prime(o))
                delta_w_1 = np.matmul(self.w2,delta_w_2)
                delta_w_1= self.sigmoid_prime(h[:-1])*delta_w_1[:-1]
                delta_w_1 = self.etha*np.outer(self.input[i],delta_w_1)
                delta_w_2 = self.etha*np.outer(h,delta_w_2)
                #sum delta weights all cases
                sum1 =sum1 + delta_w_1
                sum2 =sum2 + delta_w_2
                sum_Error += Error
                
                #print(net,h,o_net,o)
                #print(delta_w_2,"\n",delta_w_1)
                
            #update weight
            self.w1 = self.w1 - sum1
            self.w2 = self.w2 - sum2
            
            #current iteration, error, time
            if (verbose == True):
                if (j%5000) == 0 or (j == iteration):
                    print("iteration: %d/%d, Error = %f, time: %f"%(j,iteration,sum_Error,time.time()-t))
            #Error - iteration graph
            if Error_graph == True:
                y.append(sum_Error)
        if Error_graph == True:
            plt.plot(np.linspace(1,iteration,iteration),y,'black')
            plt.title("Error by iteration",size=20)
            plt.xlabel('Iteration',size=20)
            plt.ylabel('Error',size=20)
            plt.show()
    def feed_forward(self,x):
        bias = np.ones(shape=(1))
        x = np.concatenate([x,bias])
        net = np.matmul(x,self.w1)
        h = self.sigmoid(net)
        h = np.concatenate([h,np.ones(1).T])    
        o_net = np.matmul(h,self.w2)
        o = self.sigmoid(o_net)
        return o
        

In [20]:
''' 
problem 3
'''
'''
a = [[1,1],[1,0],[0,1],[0,0]]
b= [[0],
    [1],
    [1],
    [0]]
bp = BPNN(etha = 0.5 , nodes= 2 , inputs=a, outputs=b)
bp.run(5000,verbose = True,Error_graph = False)
bp.w1
'''

iteration: 5000/5000, Error = 0.500000, time: 1.306607


array([[-0.08687985,  0.08647317],
       [ 0.01736234, -0.05429326],
       [ 0.09138746, -0.01105019]])

In [10]:
def f(x):
    return x*(1-x)

if __name__ == '__main__':
    #make a target & input
    x = np.arange(0,1.05,0.1).reshape(11,1)
    target = f(x)
    #NN create
    quadratic = BPNN(0.7, 4, x, target)
    
    #NN run
    quadratic.run(500000, verbose =True, Error_graph = True)
    
    #x for graph
    a = np.linspace(0,1,100)
    x = []
    for i in a:
        x.append(quadratic.feed_forward([i])[0])
    #Graph 
    figure = plt.figure()
    plt.plot(a,x,'r')
    plt.plot(np.linspace(0,1,11),target,'b')
    plt.title("Result",size=20)
    plt.legend(['train','target'],loc=2,prop={'size': 20})
    plt.xlabel('Input',size=20)
    plt.ylabel('Output',size=20)
    plt.show()
    #Print final weight
    print(quadratic.w1)
    print(quadratic.w2)

iteration: 5000/500000, Error = 0.042895, time: 3.337077
iteration: 10000/500000, Error = 0.042882, time: 6.679143
iteration: 15000/500000, Error = 0.042861, time: 9.987332
iteration: 20000/500000, Error = 0.042869, time: 13.289505
iteration: 25000/500000, Error = 0.042959, time: 16.617569
iteration: 30000/500000, Error = 0.042946, time: 19.938726
iteration: 35000/500000, Error = 0.042809, time: 23.249840
iteration: 40000/500000, Error = 0.042899, time: 26.556998
iteration: 45000/500000, Error = 0.042405, time: 29.886098
iteration: 50000/500000, Error = 0.040032, time: 33.206218
iteration: 55000/500000, Error = 0.041307, time: 36.509390
iteration: 60000/500000, Error = 0.030768, time: 39.809567
iteration: 65000/500000, Error = 0.015627, time: 43.096816
iteration: 70000/500000, Error = 0.004691, time: 46.385982
iteration: 75000/500000, Error = 0.004607, time: 49.682206
iteration: 80000/500000, Error = 0.004442, time: 52.975403
iteration: 85000/500000, Error = 0.003706, time: 56.280531
i

In [9]:
    a = np.linspace(0,1,100)
    x = []
    for i in a:
        x.append(quadratic.feed_forward([i])[0])
    #Graph 
    figure = plt.figure()
    plt.plot(a,x,'r')
    plt.plot(np.linspace(0,1,11),target,'b')
    plt.title("Result",size=20)
    plt.legend(['train','target'],loc=2,prop={'size': 20})
    plt.xlabel('Input',size=20)
    plt.ylabel('Output',size=20)
    plt.show()